## 招商信用卡_消费金融场景下的用户购买预测

比赛网址: http://www.datafountain.cn/?u=7610684&&#/competitions/287/intro  

In [1]:
import pandas as pd # to import csv and for data manipulation
import matplotlib.pyplot as plt # to plot graph
import seaborn as sns # for intractve graphs
import numpy as np # for linear algebra
import datetime # to dela with date and time
%matplotlib inline
from sklearn.preprocessing import StandardScaler # for preprocessing the data
from sklearn.ensemble import RandomForestClassifier # Random forest classifier
from sklearn.tree import DecisionTreeClassifier # for Decision Tree classifier
from sklearn.svm import SVC # for SVM classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split # to split the data
from sklearn.model_selection import KFold # For cross vbalidation
from sklearn.model_selection import GridSearchCV # for tunnig hyper parameter it will use all combination of given parameters
from sklearn.model_selection import RandomizedSearchCV # same for tunning hyper parameter but will use random combinations of parameters
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve,roc_auc_score,classification_report
from sklearn.metrics import roc_auc_score
import warnings
import time
import datetime
import math
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
warnings.filterwarnings('ignore')
basepath="/home/linzhenpeng/Documents/python_workspace/pycharm/project/招商银行/"
datapath=basepath+"data/"

## 训练

In [2]:
def train_predict(data):
    time_date = time.strftime('%Y-%m-%d',time.localtime(time.time()))
    train = data[data['FLAG']!=-1]
    test = data[data['FLAG']==-1]
    
    train=train.fillna(0)
    test=test.fillna(0)
    
    
#     os = SMOTE(random_state=0)

#     data_train_X=train.ix[:,train.columns != "FLAG"]
#     data_train_y=train.ix[:,train.columns == "FLAG"]
#     columns = data_train_X.columns
#     os_data_X,os_data_y=os.fit_sample(data_train_X,data_train_y)
#     os_data_X = pd.DataFrame(data=os_data_X,columns=columns)
#     os_data_y= pd.DataFrame(data=os_data_y,columns=["FLAG"])
#     train2=pd.concat([os_data_X,os_data_y],axis=1)
#     train2=train2[train2["FLAG"]==1].sample(300)
# #     train_0=train[train["FLAG"]==0].sample((int)(train.shape[0]*0.8))
# #     train_1=train[train["FLAG"]==1]
#     train=pd.concat([train,train2],axis=0)
#     train=train.sample(frac=1).reset_index(drop=True)    
    
    train_userid = train.pop('USRID')
    y = train.pop('FLAG')
    col = train.columns
    X = train[col].values


    test_userid = test.pop('USRID')
    test_y = test.pop('FLAG')
    test = test[col].values

    N = 5
    skf = StratifiedKFold(n_splits=N,shuffle=False,random_state=1314)

    xx_cv = []
    xx_pre = []
    df_imp=[]

    for train_in,test_in in skf.split(X,y):
        X_train,X_test,y_train,y_test = X[train_in],X[test_in],y[train_in],y[test_in]
        print(X_train.shape)
        print(X_test.shape)
        # create dataset for lightgbm
        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

        # specify your configurations as a dict
#         params = {
#             'boosting_type': 'gbdt',
#             'objective': 'binary',
#             'metric': {'auc'},
#             'num_leaves': 32,
#             'learning_rate': 0.01,
#             'feature_fraction': 0.9,
#             'bagging_fraction': 0.8,
#             'bagging_freq': 5
# #             'verbose': 0
#         }
        params = {
               'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': {'auc'},
                'num_leaves':21,
                'max_depth':3,
                'max_bin':20,
                'min_data_in_leaf':95,
                'feature_fraction': 0.3,
                'bagging_fraction':  0.8,
                'bagging_freq': 20,
                'lambda_l1':0.7,
                'lambda_l2':0.6,
                'min_split_gain':1.0
                }

        print('Start training...')
        # train
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=40000,
                        valid_sets=lgb_eval,
                        verbose_eval=250,
                        early_stopping_rounds=50)

        # print('Save model...')
        # save model to file
        # gbm.save_model('model.txt')

        print('Start predicting...')
        y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
        xx_cv.append(roc_auc_score(y_test,y_pred))
        xx_pre.append(gbm.predict(test, num_iteration=gbm.best_iteration))
        
        
#         df = pd.DataFrame(train.columns.tolist(), columns=['feature'])
#         df['importance']=list(gbm.feature_importance())                           # 特征分数
#         df = df.sort_values(by='importance',ascending=False)                      # 特征排序
#         df.to_csv(basepath+'feature/feature_score_%s.csv'%(str(time_date)),index=None,encoding='gbk')  # 保存分数
#         df_imp=df[df["importance"]!=0].feature.values
#         print(df_imp)
    
#     data=data.ix[:,df_imp]
#     train_predict(data,True)

    s = 0
    for i in xx_pre:
        s = s + i

    s = s /N

    res = pd.DataFrame()
    res['USRID'] = list(test_userid.values)
    res['RST'] = list(s)

    print('xx_cv',np.mean(xx_cv))

    res.to_csv(basepath+'submit/%s_%s.csv'%(str(time_date),str(np.mean(xx_cv)).split('.')[1]),index=False,sep='\t')
    return res

## 特征

In [3]:
# 读取个人信息
train_agg = pd.read_csv(datapath+'train_agg.csv',sep='\t')
test_agg = pd.read_csv(datapath+'test_agg.csv',sep='\t')
agg = pd.concat([train_agg,test_agg],copy=False)

# 日志信息
train_log = pd.read_csv(datapath+'train_log.csv',sep='\t')
test_log = pd.read_csv(datapath+'test_log.csv',sep='\t')
log = pd.concat([train_log,test_log],copy=False)

# 用户唯一标识
train_flg = pd.read_csv(datapath+'train_flg.csv',sep='\t')
test_flg = pd.read_csv(datapath+'submit_sample.csv',sep='\t')
test_flg['FLAG'] = -1
del test_flg['RST']
flg = pd.concat([train_flg,test_flg],copy=False)

data = pd.merge(agg,flg,on=['USRID'],how='left',copy=False)

In [4]:
log['time']=log['OCC_TIM'].apply(lambda x:time.strptime(x, "%Y-%m-%d %H:%M:%S"))
log['mktime'] = log['time'].apply(lambda x:time.mktime(x))
log['days']=log["OCC_TIM"].apply(lambda x:x.split(" ")[0])
log['hours']=log["OCC_TIM"].apply(lambda x:x.split(" ")[1].split(":")[0])
log['EVT_LBL_1'] = log['EVT_LBL'].apply(lambda x:x.split('-')[0])
log['EVT_LBL_2'] = log['EVT_LBL'].apply(lambda x:x.split('-')[1])
log['EVT_LBL_3'] = log['EVT_LBL'].apply(lambda x:x.split('-')[2])
log['EVT_LBL_12'] = log['EVT_LBL'].apply(lambda x:x.split('-')[0]+"-"+x.split('-')[1])

log_l1=log[log["days"]=="2018-03-31"]
log_l2=log[log["days"]>="2018-03-30"]
log_l3=log[log["days"]>="2018-03-29"]
log_l5=log[log["days"]>="2018-03-27"]
log_l7=log[log["days"]>="2018-03-25"]
log_l10=log[log["days"]>="2018-03-21"]
log_l15=log[log["days"]>="2018-03-15"]
log_l20=log[log["days"]>="2018-03-10"]
log_p3=log[log["days"]<="2018-03-03"]
log_p7=log[log["days"]<="2018-03-07"]
log_p10=log[log["days"]<="2018-03-10"]


d1 = datetime.datetime(2018,4,1)
log["hd"]=log["time"].apply(lambda x:(d1-datetime.datetime(x.tm_year,x.tm_mon,x.tm_mday)).days)

In [69]:
log = log.sort_values(['USRID','mktime'])
log['next_time'] = log.groupby(['USRID'])['mktime'].diff(-1).apply(np.abs)
log_next_time = log.groupby(['USRID'],as_index=False)['next_time'].agg({
    'next_time_mean':np.mean,
    'next_time_std':np.std,
    'next_time_min':np.min,
    'next_time_max':np.max
})
log_next_time["have_log"]=1
data2=pd.merge(data,log_next_time,on=["USRID"],how='left')

In [70]:
#用户总点击
user_total_count=log.groupby("USRID").agg({"USRID":"count"})
data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_count'))

# user_total_count=log_l1.groupby("USRID").agg({"USRID":"count"})
# data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_l1_count'))

# user_total_count=log_l2.groupby("USRID").agg({"USRID":"count"})
# data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_l2_count'))

user_total_count=log_l3.groupby("USRID").agg({"USRID":"count"})
data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_l3_count'))

# user_total_count=log_l5.groupby("USRID").agg({"USRID":"count"})
# data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_l5_count'))

user_total_count=log_l7.groupby("USRID").agg({"USRID":"count"})
data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_l7_count'))

# user_total_count=log_l10.groupby("USRID").agg({"USRID":"count"})
# data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_l10_count'))

# user_total_count=log_l15.groupby("USRID").agg({"USRID":"count"})
# data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_l5_count'))

# user_total_count=log_l20.groupby("USRID").agg({"USRID":"count"})
# data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_l20_count'))

# user_total_count=log_p3.groupby("USRID").agg({"USRID":"count"})
# data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_p3_count'))

# user_total_count=log_p7.groupby("USRID").agg({"USRID":"count"})
# data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_p7_count'))

# user_total_count=log_p10.groupby("USRID").agg({"USRID":"count"})
# data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_user_total_p10_count'))



In [71]:
#各天的活跃次数以及权重
user_count=pd.crosstab(log["USRID"],log["days"])
user_count1=user_count.copy()
user_count["max"]=user_count1.max(axis=1)
user_count["act_days"]=(user_count1 != 0).sum(axis=1)
user_count["std"]=user_count1.std(axis=1)
user_count["mean"]=user_count1.mean(axis=1)
user_count1=user_count1.apply(lambda x:x*math.exp(int(x.name.split("-")[2])))
user_count['day_all_weight']=user_count1.sum(axis=1)
# user_count["have_log"]=1
# user_count=user_count.ix[:,["day_all_weight","USRID","max","act_days","std","mean","have_log"]]
data2=pd.merge(data2,user_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_day_user_count'))

user_count=pd.crosstab(log_l3["USRID"],log_l3["days"])
user_count1=user_count.copy()
user_count["max_3"]=user_count1.max(axis=1)
user_count["act_days_3"]=(user_count1 != 0).sum(axis=1)
user_count["std_3"]=user_count1.std(axis=1)
user_count["mean_3"]=user_count1.mean(axis=1)
user_count1=user_count1.apply(lambda x:x*math.exp(int(x.name.split("-")[2])-25))
user_count['day_3_weight']=user_count1.sum(axis=1)
user_count=user_count.ix[:,["day_3_weight","USRID","max_3","act_days_3","std_3","mean_3"]]
data2=pd.merge(data2,user_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_day_user_l3_count'))

user_count=pd.crosstab(log_l7["USRID"],log_l7["days"])
user_count1=user_count.copy()
user_count["max_7"]=user_count1.max(axis=1)
user_count["act_days_7"]=(user_count1 != 0).sum(axis=1)
user_count["std_7"]=user_count1.std(axis=1)
user_count["mean_7"]=user_count1.mean(axis=1)
user_count1=user_count1.apply(lambda x:x*math.exp(int(x.name.split("-")[2])-20))
user_count['day_7_weight']=user_count1.sum(axis=1)
user_count=user_count.ix[:,["day_7_weight","USRID","max_7","act_days_7","std_7","mean_7"]]
data2=pd.merge(data2,user_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_day_user_l7_count'))

# user_count=pd.crosstab(log_p7["USRID"],log_p7["days"])
# user_count1=user_count.copy()
# user_count["max_p7"]=user_count1.max(axis=1)
# user_count["act_days_p7"]=(user_count1 != 0).sum(axis=1)
# user_count["std_p7"]=user_count1.std(axis=1)
# user_count["mean_p7"]=user_count1.mean(axis=1)
# user_count1=user_count1.apply(lambda x:x*math.exp(int(x.name.split("-")[2])))
# user_count['day_p7_weight']=user_count1.sum(axis=1)
# user_count=user_count.ix[:,["day_p7_weight","USRID","max_p7","act_days_p7","std_p7","mean_p7"]]
# data2=pd.merge(data2,user_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_day_user_p7_count'))


In [72]:
#用户在不同设备上的点击次数
tch_type=pd.crosstab([log["USRID"]],log["TCH_TYP"],dropna=False)
data2 = pd.merge(data2,tch_type,left_on=['USRID'],how='left',right_index=True,copy=False,suffixes=('', '_tch_type'))

In [73]:
#最大点击间隔
user_last_touch=log.groupby(["USRID"])["mktime"].apply(lambda x:x.max()-x.min())
data2 = pd.merge(data2,user_last_touch.to_frame(),left_on=['USRID'],how='left',right_index=True,copy=False,suffixes=('', '_user_last_touch'))

In [74]:
#最后操作距离4月1号的天数
user_last_touch=log.groupby(["USRID"])["hd"].apply(lambda x:x.min())
data2 = pd.merge(data2,user_last_touch.to_frame(),left_on=['USRID'],how='left',right_index=True,copy=False,suffixes=('', '_user_last_day'))

In [75]:
lbl_1=pd.crosstab(log["USRID"],log["EVT_LBL_1"])
data2=pd.merge(data2,lbl_1,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_lbl_1'))
lbl_2=pd.crosstab(log["USRID"],log["EVT_LBL_2"])
data2=pd.merge(data2,lbl_2,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_lbl_2'))
# lbl_3=pd.crosstab(log["USRID"],log["EVT_LBL_3"])
# data2=pd.merge(data2,lbl_3,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_lbl_3'))

In [76]:
#重点页面处理1724,3614,4285,3744,3768,3765          
#1700,1703,1678,3743,1718
# hot_lbl_3=['1724','3614','4285','3744','3768','3765','1700','1703','1678','3743','1718']
hot_lbl_3=['1724','3614','4285','3744','3768','3765','1700','1703','1678','3743','1718',
           '117','3640','3766','4301','4311','1725','573','1722','565','277','4357','558',
           '1523','3862','2637','564','1710','3769','3622','3248','3623','1677','914','2206',
           '3751','3619','2640','3838','3864','3807','1525','2004','4282','2005','2003','559',
           '2078','1197','2044','2204','1196','4329','562','3772','3494','3620','566','910',
           '3767','4300','1694','1699','4325','17','4303','3771','2052','3834','3121','4326',
           '3843','4391','891','3830','1711','3929','4309','3845','4302','19','1712','1728',
           '1726','3795','3759','3847','1709','3927','2207','4376','3496','3498','2074',
           '4278','3617','1707','43','2043','1708','3745','1701','3819','3801','3813','3805',
           '1704','3828','4307','1723','3794','3839','3774','3867','2644','2050','3806',
           '2058','2062','3615','2073','2205','3808','1729','4366','462','561','912','911',
           '905','897','4378','1679','1693']
log_hot_lbl_3=log.query("EVT_LBL_3 in "+str(hot_lbl_3))

user_total_count=log_hot_lbl_3.groupby("USRID").agg({"USRID":"count"})
data2=pd.merge(data2,user_total_count,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_hot_lbl_3_count'))

log_next_time = log_hot_lbl_3.groupby(['USRID'],as_index=False)['next_time'].agg({
    'hot_3_next_time_mean':np.mean,
    'hot_3_next_time_std':np.std,
    'hot_3_next_time_min':np.min,
    'hot_3_next_time_max':np.max
})
data2=pd.merge(data2,log_next_time,on=["USRID"],how='left')

user_last_touch=log_hot_lbl_3.groupby(["USRID"])["hd"].apply(lambda x:x.min())
data2 = pd.merge(data2,user_last_touch.to_frame(),left_on=['USRID'],how='left',right_index=True,copy=False,suffixes=('', '_hot_3_day'))

lbl_3=pd.crosstab(log_hot_lbl_3["USRID"],log_hot_lbl_3["EVT_LBL_3"])
data2=pd.merge(data2,lbl_3,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_hot_lbl_3'))



In [80]:
lbl_1=pd.crosstab(log_l3["USRID"],log_l3["EVT_LBL_1"])
data2=pd.merge(data2,lbl_1,left_on=["USRID"],right_index=True,how="left",suffixes=('', '_lbl_3_1'))

In [77]:
data2["V7_0"]=data2["V7"]+0.02653405368112591
data2["V7_1"]=data2["V7"]-0.6554727928211695

In [40]:
data.ix[data["FLAG"]==0,"V17"].mean()

-0.012931384983863033

## 运行

In [78]:
data2.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,573,891,897,905,910,911,912,914,V7_0,V7_1
0,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,-1.70618,-0.29641,-0.18761,-0.48351,-0.53051,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,-0.269876,-0.951883
1,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,-1.70618,0.37601,-0.10030,-0.16694,-0.31897,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.402544,-0.279463
2,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,-1.57176,0.03980,-0.17089,-0.38610,-0.46000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.066334,-0.615673
3,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,-1.30291,-0.29641,-0.18761,-0.45916,-0.53051,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,-0.269876,-0.951883
4,-1.92554,-0.90689,-1.26634,0.2892,-0.68454,-1.16849,-0.29641,-0.18761,-0.50786,-0.60103,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.269876,-0.951883


In [81]:
result_3=train_predict(data2)

(63999, 452)
(16001, 452)
Start training...
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[154]	valid_0's auc: 0.853434
Start predicting...
(64000, 452)
(16000, 452)
Start training...
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.860501
Early stopping, best iteration is:
[203]	valid_0's auc: 0.860893
Start predicting...
(64000, 452)
(16000, 452)
Start training...
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[181]	valid_0's auc: 0.857495
Start predicting...
(64000, 452)
(16000, 452)
Start training...
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's auc: 0.860211
Early stopping, best iteration is:
[237]	valid_0's auc: 0.860456
Start predicting...
(64001, 452)
(15999, 452)
Start training...
Training until validation scores don't improve for 50 rounds.
Early stopping, best iteration is:
[138]	valid_0's auc: 0.851884
Start